In [5]:
# Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install beautifulsoup4

You should consider upgrading via the 'C:\Users\calep\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [44]:
import json, re, ssl
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [45]:
BASE_URL = "https://www.indeed.com/"

#for ignoring ssl certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [46]:
def scrape_data(url_to_scrape, number_of_pages_nos):
    """This function loops through the number of pages of search results that need to be scraped and passes the data into
    another function to extract particular data points for each of the the inividial job post in a single page of a job post result
    """
    data_collected = []
    for i in range(0, number_of_pages_nos):
        extension = ""
        if i != 0:
            extension = "&start=" + str(i*10)
        url = url_to_scrape + extension
        req = Request(url, headers={"User-Agent": "Mozilla\5.0"})
        web_page = urlopen(req).read()
        soup = BeautifulSoup(web_page, "html.parser")
        # print(soup)
        data_collected = get_data_from_webpage(data_collected, soup)
    return data_collected


In [66]:
def extract_data_points(job, div):
    """
    This function helps extract specific data points such as title, companyName, rating and more for each job post in a search result page
    """
    for elem in div.findAll('h2', attrs={'class': 'jobTitle'}):
        job['title'] = elem.text

    for elem in div.findAll('span', attrs={'class': 'companyName'}):
        job['companyName'] = elem.text

    for elem in div.findAll('div', attrs={'class': 'companyLocation'}):
        job['companyLocation'] = elem.text


    print(job)

    return job

In [67]:
def get_data_from_webpage(data_collected, soup):
    """
    This function loops through the different job posts in a single page of search results and extracts information for each individual job post
    """
    job_posts = []
    for div in soup.findAll('a', attrs={'class': 'result'}):

        # print("found a new div! \n {}".format(div))

        job = dict()
        job = extract_data_points(job, div)
        job_posts.append(div['data-jk'])
        single_job_post_extension_url = "https://www.indeed.com/viewjob?jk=" + div["data-jk"]
        job["url"] = single_job_post_extension_url
        req = Request(single_job_post_extension_url, headers = {"User-Agent": "Mozilla/5.0"})
        web_page = urlopen(req).read()
        job_soup = BeautifulSoup(web_page, 'html.parser')

        for inside_div in job_soup.findAll('div', attrs={"class": "jobsearch-jobDescriptionText"}):
            details = inside_div.text
            job["details"] = re.sub(' +', ' ', details.replace("\n", " "))
            
        data_collected.append(job)
    return data_collected

In [68]:
def main(name_of_city, keywords, number_of_pages_nos):
    """main function where inputs are taken and the first url we use is constructed.
    Also, the final JSON is saved here using the data scraped.  """
    # name_of_city = input("Enter the name of city you want to search- ")
    # keywords = input("Enter your desired search keywords")
    # number_of_pages = input("Enter the number of pages of search results that you want to scrape for provided keywords- ")

    name_of_city = '+'.join(name_of_city.split(' '))
    keywords = '+'.join(keywords.split(' '))

    url_to_scrape = BASE_URL + "/jobs?q=" + keywords + "&1-" + name_of_city
    print("the url to scrape is {}".format(url_to_scrape))

    data_collected = scrape_data(url_to_scrape, number_of_pages_nos)
    with open('data.json', 'w') as fp:
        json.dump(data_collected, fp, sort_keys=True, indent=4, ensure_ascii=False)

In [69]:
main("Milwaukee", "Web Developer", 3)

the url to scrape is https://www.indeed.com//jobs?q=Web+Developer&1-Milwaukee
{'title': 'Front End Web Developer', 'companyName': 'Image Management', 'companyLocation': 'Racine, WI 53403'}
{'title': 'Front End Developer', 'companyName': 'Triple Crown Products', 'companyLocation': 'Mukwonago, WI 53149'}
{'title': 'Web Developer Milwaukee', 'companyName': 'OhmzTech', 'companyLocation': 'Milwaukee, WI'}
{'title': 'Web Front End React Developer', 'companyName': 'Assemble Inc.', 'companyLocation': 'Remote+1 location'}
{'title': 'Front-End developer (remote)', 'companyName': 'OpenClassrooms', 'companyLocation': 'Chicago, IL+10 locations•Remote'}
{'title': 'Web Designer/Developer', 'companyName': 'Chicago Advertising Agency', 'companyLocation': 'Remote'}
{'title': 'eCommerce Web Developer - JavaScript Focus', 'companyName': 'oBundle', 'companyLocation': 'Remote'}
{'title': 'Javascript Developer', 'companyName': 'ion Software Group', 'companyLocation': 'Pleasant Prairie, WI'}
{'title': 'Shopif